In [33]:
import os
import pandas as pd
import numpy as np
from datetime import (
    datetime
)
from dateutil.relativedelta import relativedelta

from table_wrangle_functions import (
    add_datestamp
)

# Ensure the 'outputs' directory exists
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

# Load the data
adhd_medication_data = pd.read_csv("/workspaces/ADHD-Pre-and-Post-Covid/output/Patient_table_3_rolling_6month_incident.csv.gz")

# getting the month and year only
adhd_medication_data['last_mph_med_date'] = (
    pd.to_datetime(adhd_medication_data['last_mph_med_date'], format='%Y-%m-%d')
)

#This is a crude way of removing the day
adhd_medication_data['last_mph_med_date_month_date'] = (
    adhd_medication_data['last_mph_med_date'].apply(lambda x: x.strftime('%Y-%m'))
)

adhd_medication_data['last_mph_med_date_month_date'] = (
    pd.to_datetime(adhd_medication_data['last_mph_med_date_month_date'], format='%Y-%m')
)

output = (
    adhd_medication_data
            .groupby(['last_mph_med_date_month_date','age_band','sex'],as_index=False).size()
)


In [37]:
from datetime import (
    datetime
)

from dateutil import relativedelta

def monthly_interval_list(start_date, end_date):
    
    #Differnce in months
    delta = relativedelta.relativedelta(end_date, start_date)

    res_months = delta.months + (delta.years * 12)

    interval_array = [start_date + relativedelta.relativedelta(months=each_month) for each_month in range(res_months+1)]

    return interval_array




In [74]:
dataframe = output.copy()
date_column = 'last_mph_med_date_month_date'
count_column = 'size'
column_group = ['age_band','sex']
rolling_col_name = 'rolling'



In [ ]:
#Need to get inputs
date_interval = [min(dataframe[date_column]), max(dataframe[date_column])]
unqiue_combination = dataframe[column_group].drop_duplicates(subset=column_group)

list_of_dates = monthly_interval_list(date_interval[0], date_interval[1])
list_of_dates.sort()

#Create an empty dataframe with the dates
rolling_template = pd.DataFrame( {date_column:list_of_dates})

output_table_with_rolling = pd.DataFrame() 

#Need to get the vaules of each 
for each_index in unqiue_combination.index:

    each_conbination = unqiue_combination.loc[[each_index],:]

    each_conbination_dict = each_conbination.to_dict('records')[0]

    # Getting the filtered vaules - Source: https://stackoverflow.com/questions/34157811/filter-a-pandas-dataframe-using-values-from-a-dict
    filtred_group = dataframe.loc[(dataframe[list(each_conbination_dict)] == pd.Series(each_conbination_dict)).all(axis=1)]

    #This is to for the join
    cobination_to_join = filtred_group[[date_column,count_column]]

    #This is join and fill in the missing vaule
    each_roll = rolling_template.merge(
        cobination_to_join,
        how='left',
        left_on=date_column,
        right_on=date_column
    ).fillna(0)

    #Sort 
    each_roll = each_roll.sort_values(by=date_column)

    #Need to the 6 month rolling
    each_roll[rolling_col_name] = each_roll[count_column].rolling(6,min_periods = 1).sum()

    # Add back the columns and values from each_conbination to each_roll
    for col in each_conbination.columns:
        each_roll[col] = each_conbination.iloc[0][col]

    # Concatenate each_roll into empty_df
    output_table_with_rolling = pd.concat([output_table_with_rolling, each_roll], ignore_index=True)



In [80]:
output_table_with_rolling

,last_mph_med_date_month_date,size,rolling,age_band,sex
0,2016-03-01,1.0,1.0,0 to 9,female
1,2016-04-01,0.0,1.0,0 to 9,female
2,2016-05-01,0.0,1.0,0 to 9,female
3,2016-06-01,0.0,1.0,0 to 9,female
4,2016-07-01,0.0,1.0,0 to 9,female
...,...,...,...,...,...
219,2025-02-01,33.0,113.0,0 to 9,male
220,2025-03-01,37.0,141.0,0 to 9,male
221,2025-04-01,43.0,166.0,0 to 9,male
222,2025-05-01,50.0,198.0,0 to 9,male


In [60]:
cobination_to_join

,last_mph_med_date_month_date,size
0,2016-03-01,1
2,2016-11-01,1
12,2018-05-01,1
13,2018-06-01,1
14,2018-09-01,1
...,...,...
130,2025-02-01,23
132,2025-03-01,21
134,2025-04-01,36
136,2025-05-01,36
